# Algunas cuentitas para la rep $6$ de $SU(3)$ _en la base de pesos_

Queremos calcular los momentos y windings de todos los estados para eventualmente calcular las $\tilde{f}_{abc}$ que aparecen en las derivadas covariantes y variaciones de interpolation.

Tenemos todos los estados en la base de pesos y en la base de raíces, pero para poder hacer momentos y windings necesitamos calcular los momentos right. Para hacerlo, primero calculamos usando la LMC el cuadrado del momento right.

de LMC sabemos que

$\frac{1}{2}\left(l^{m}_{L}\alpha_{m}\right)^{2} - \frac{1}{2}\left(l^{m}_{R}\alpha_{m}\right)^{2} = 1- N$

donde $N$ es el número de osciladores left (0,1), y $l^{m}_{L}$ son las componentes de cada estado en la base de raíces simples (análogamente para el right).

Ahora pasemos a la base de pesos.

Si llamamos $(l_{1L},l_{2L}) = (a,b)$ a dichas componentes por simplicidad, puede verse que la expresión anterior se reescribe como

$\left(l_{mR} \omega_{m} \right)^{2} = \frac{2}{3}(a^{2}  + b^{2} + ab) - 2$

para los estados $s = 1,3,6$ con $N_{s}=0$, y 

$\left(l_{mR} \omega_{m} \right)^{2} =  \frac{2}{3}(a^{2}  + b^{2} + ab)$

para los estados $s = 2,4,5$ con $N_{s}=1$. 

In [7]:
-- Function to calculate (2/3)*(a^2 b^2 + ab) - 2 i.e. right momentum squared for charged scalars
-- I multiply by 3 to avoid getting decimals, the result should be 2/3, so with this modification it goes to 2.
calcLR1_3_6 :: (Double, Double) -> Double
calcLR1_3_6 (a, b) = 
  2*(a^2 + b^2 + a*b)- 6

-- Function to calculate (2/3)*(a^2 + b^2 + a*b) i.e. right momentum squared for cartan scalars
calcLR2_4_5 :: (Double, Double) -> Double
calcLR2_4_5 (a, b) = 
  2*(a^2 + b^2 + a*b) 

main1 :: IO ()
main1 = do
  let results = [
        ("LR1", calcLR1_3_6 (2,0), (2,0)),
        ("LR3", calcLR1_3_6 (-2,2), (-2,2)),
        ("LR6", calcLR1_3_6 (0,-2), (0,-2)),
        ("LR2", calcLR2_4_5 (0,1), (0,1)),
        ("LR4", calcLR2_4_5 (1,-1), (1,-1)),
        ("LR5", calcLR2_4_5 (-1,0), (-1,0))
         ]
  
  mapM_ (\(label, value, tuple) -> putStrLn $ label ++ " = " ++ show value ++ " for " ++ show tuple) results


In [8]:
main1

LR1 = 2.0 for (2,0)
LR3 = 2.0 for (-2,2)
LR6 = 2.0 for (0,-2)
LR2 = 2.0 for (0,1)
LR4 = 2.0 for (1,-1)
LR5 = 2.0 for (-1,0)

Sabiendo cuánto valen los momentos right cuadrados, podemos ahora buscar cuáles son los valores $l^{m}_{R}$, que podrían o no ser todos iguales para los 6 estados (con esta cuenta no basta para discernir más que aquellos plausibles). Si llamamos $(x,y,z) = (l^{1}_{R},l^{2}_{R})$, vemos que las componentes en la base de pesos fundamentales del momento right deben satisfacer

$\frac{2}{3}(x^2 + y^2 + xy)= \frac{2}{3}$,

that is,

$(x^2 + y^2 + xy)= 1$

In [9]:
-- Haskell program to find integer solutions (x, y) to the equation:
-- (x^2 + y^2 + xy)= 1


-- Function to check if the given (x, y) satisfies the equation
satisfiesEquation :: Integer -> Integer -> Bool
satisfiesEquation x y = x^2 + y^2 + x*y == 1

-- Function to generate all integer numbers within a given range
generateIntegers :: Integer -> Integer -> [Integer]
generateIntegers start end = [n | n <- [start..end]]

-- Main function to find the first three solutions
findSolutions :: Integer -> Integer -> [(Integer, Integer)]
findSolutions start end = take 20 [(x, y) | x <- integers, y <- integers, satisfiesEquation x y]
  where
    integers = generateIntegers start end

-- Print the solutions
main2 :: IO ()
main2 = do
    let solutions = findSolutions (-10) 10
    mapM_ print solutions


Line 14: Redundant list comprehension
Found:
[n | n <- [start .. end]]
Why not:
[start .. end]

In [10]:
main2

(-1,0)
(-1,1)
(0,-1)
(0,1)
(1,-1)
(1,0)

In [ ]:
-- Import necessary modules
import Control.Monad (guard)

-- Function to find solutions
findSolutions :: [(Integer, Integer)] -> [(Int, Int, Int, Int, (Integer, Integer))]
findSolutions targets = do
    -- Define the range for variables
    n1 <- [-10..10]
    n2 <- [-10..10]
    w1 <- [-10..10]
    w2 <- [-10..10]
    
    -- Calculate the left and right equations
    let l1L = fromIntegral (w1 + n1)
        l2L = fromIntegral (-w1+w2+n2)
        
        l1R = fromIntegral (-w1+w2+n1)
        l2R = fromIntegral (-w2+n2)
       
    -- Check the right equations
    guard (l1R == -1 && l2R == 0)
    -- this one corresponds to the one used on interpolation, which is [q_r]_\alpha=(-2/3, -1/3) in the root basis.
    -- For SU(3), we have that \alpha_1 = 2\omega_1 - \omega_2; \alpha_2=-\omega_1 + 2\omega_2

    -- Check if any of the left equations match the target tuples
    target <- targets
    guard (l1L == fst3 target && l2L == snd3 target)

    -- Return the solution tuple along with the target
    return (n1, n2, w1, w2, target)
  where
    fst3 (x, _) = x
    snd3 (_, y) = y
    
-- Main function to print the solutions
main3 :: IO ()
main3 = do
    -- Define the target tuples for the left equations
    let targets = [ (2,0)
                  , (0,1)
                  , (-2,2)
                  , (1,-1)
                  , (-1,0)
                  , (0,-2)
                  ]

    -- Find and print the solutions
    let solutions = take 10 $ findSolutions targets
    mapM_ printSolution solutions
  where
    -- Function to print each solution with its corresponding target
    printSolution (n1, n2, w1, w2, target) = do
        putStrLn $ "Solution: (n1, n2, w1, w2) = " ++ show (n1, n2, w1, w2)
        putStrLn $ "Corresponding to left equation results: " ++ show target
        putStrLn ""



In [ ]:
main3

Solution: (n1, n2, w1, w2) = (-2,1,0,1)
Corresponding to left equation results: (-2,2)

Solution: (n1, n2, w1, w2) = (-1,0,0,0)
Corresponding to left equation results: (-1,0)

Solution: (n1, n2, w1, w2) = (-1,1,1,1)
Corresponding to left equation results: (0,1)

Solution: (n1, n2, w1, w2) = (0,-1,0,-1)
Corresponding to left equation results: (0,-2)

Solution: (n1, n2, w1, w2) = (0,0,1,0)
Corresponding to left equation results: (1,-1)

Solution: (n1, n2, w1, w2) = (0,1,2,1)
Corresponding to left equation results: (2,0)